<a href="https://colab.research.google.com/github/adityakalkeri1/Projects/blob/NLP/Tweet_analysis_Naive_Bayes%2C_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uploading Dataset to Colab

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [7]:
!ls '/content/gdrive/MyDrive/Datasets/'

 archive.zip  'GoIbibo India Hotels'  'NYC airbnb'  'Uber dataset'


In [8]:
!unzip 'gdrive/MyDrive/Datasets/archive.zip' -d 'Archive/'

Archive:  gdrive/MyDrive/Datasets/archive.zip
replace Archive/training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Archive/training.1600000.processed.noemoticon.csv  


# Text Data Preprocessing


## 1. Cleaning the text

In [9]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('Archive/training.1600000.processed.noemoticon.csv', encoding = 'latin', header=None)

In [11]:
df

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [12]:
df.columns = ['Sentiment', 'ID', 'Date', 'Query', 'user', 'Text']

In [13]:
df.drop(['ID', 'Date', 'Query', 'user'], axis =1, inplace = True)

In [14]:
df.head(3)

,Sentiment,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...


In [15]:
(df['Text'] == 1).sum()

0

In [16]:
for i in range(10):
  print(df['Text'][i])

@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
my whole body feels itchy and like its on fire 
@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. 
@Kwesidei not the whole crew 
Need a hug 
@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?
@Tatiana_K nope they didn't have it 
@twittera que me muera ? 


In [17]:
for i in range(1080, 1090):
  print(df['Text'][i])

@onlysweeter I don't know the dance. 
@laurwee_boo im ok I guess had a hard day 
Jade is looking for a new home...  http://apps.facebook.com/dogbook/profile/view/4744827
damnnn! i missed 11:11 
@sarahprout tweetfinder hates me  and I was having trouble with background on twitter,what do you think?
@Twilighter4Life lol yeh ill be studying  stupid uni. only 2 more weeks and im on holidays! YAY! plus my birthday next week woot
@transbay &quot;SFMTA Budget Proposal Hearing: tomorrow, April 7 at 2:00 pm (City Hall, Room 400). Sadly, I cannot attend.&quot; Me neither. 
is afraid that her G.I. notes will not read themselves. 
*Sigh* Rain??? Why did you decide to show up? Move away! You were not invited to the Tuesday-party  (this is not the start I hoped for)
_secretgarden_ I haven't gotten any porn spammers  I don't check my followers, but haven't had any tweets like that.


We have to remove 'mentions' (Start with @), any websites, and numbers. We will use re to clean the data

In [18]:
import re
def text_cleaner(row):
  row = re.sub('@[a-zA-Z0-9_]+','', row)                                          #For removing usernames
  row = re.sub('(http.+\/\/[a-zA-Z.]+\/\S+|http.+\/\/[a-zA-Z0-9.]+\/)', '', row)  #For removing website names
  row = re.sub('[0-9]+', '', row)                                                 #For removing numbers
  row = re.sub('[*\(\)\?!_%-]', '', row)                                           #For removing symbols

  return row
df['Text'] = df['Text'].apply(text_cleaner)

In [19]:
for i in range(10):
  print(df['Text'][i])

   Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah
 I dived many times for the ball. Managed to save   The rest go out of bounds
my whole body feels itchy and like its on fire 
 no, it's not behaving at all. i'm mad. why am i here because I can't see you all over there. 
 not the whole crew 
Need a hug 
 hey  long time no see Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you 
 nope they didn't have it 
 que me muera  


In [20]:
for i in range(1080, 1090):
  print(df['Text'][i])

 I don't know the dance. 
 im ok I guess had a hard day 
Jade is looking for a new home...  
damnnn i missed : 
 tweetfinder hates me  and I was having trouble with background on twitter,what do you think
 lol yeh ill be studying  stupid uni. only  more weeks and im on holidays YAY plus my birthday next week woot
 &quot;SFMTA Budget Proposal Hearing: tomorrow, April  at : pm City Hall, Room . Sadly, I cannot attend.&quot; Me neither. 
is afraid that her G.I. notes will not read themselves. 
Sigh Rain Why did you decide to show up Move away You were not invited to the Tuesdayparty  this is not the start I hoped for
secretgarden I haven't gotten any porn spammers  I don't check my followers, but haven't had any tweets like that.


Now that data is cleaned, we take a look at sentiment values in the text

In [21]:
df['Sentiment'].unique()

array([0, 4])

In [22]:
labels = df['Sentiment'].copy()
texts = df['Text'].copy()

In [23]:
labels[labels == 4] = 1

Also, first 800000 rows are sentiment = 0, next are all sentiment = 1

In [24]:
df['Sentiment'][:800000].unique()

array([0])

In [25]:
df['Sentiment'][800000:].unique()

array([4])

We will shuffle the dataset, and remove the earlier index

In [27]:
data_input = [[x,y] for x,y in zip(texts, labels)]

In [28]:
import random
random.shuffle(data_input)

text = [x[0] for x in data_input]
labels = [x[1] for x in data_input]

Now we will separate labels and text, and split them into train and test data

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text, labels, random_state = 10)

# Naive Bayes and Logistic regression

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df = 5, ngram_range = (1,3)).fit(X_train)

vocab = vect.get_feature_names()

print('Length of vocab: ', len(vocab))

Length of vocab:  514454


In [31]:
X_train_vectorised = vect.transform(X_train)
X_test_vectorised = vect.transform(X_test)

In [32]:
#Now training and evaluating Naive Bayes model

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
nb_model = MultinomialNB(alpha= 0.1).fit(X_train_vectorised, y_train)
train_pred = nb_model.predict(X_train_vectorised)
test_pred = nb_model.predict(X_test_vectorised)

print('Training accuracy: ', accuracy_score(y_train,train_pred))
print()
print('Testing accuracy: ', accuracy_score(y_test,test_pred))

Training accuracy:  0.8427908333333334

Testing accuracy:  0.80269


Pretty good accuracy level, considering how simple the model is, and the training and prediction time was really quick, only 2 or 3 seconds

In [33]:
#Now training and testing logistic regression
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter = 1000).fit(X_train_vectorised, y_train)
train_pred = lr_model.predict(X_train_vectorised)
test_pred = lr_model.predict(X_test_vectorised)

print('Training accuracy: ', accuracy_score(y_train,train_pred))
print()
print('Testing accuracy: ', accuracy_score(y_test,test_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training accuracy:  0.9148441666666667

Testing accuracy:  0.8143525


Model is overfitting, plus took around 10 mins to train

# LSTM

In [ ]:
#First Tokenizing the texts

In [34]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [35]:
#Tokenizer
tokenizer = Tokenizer(num_words = 2000,oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
print('Length of vocab: ', len(word_index))

Length of vocab:  271740


In [36]:
#Converting the sentences to tokens
X_train_tokens = tokenizer.texts_to_sequences(X_train)

In [37]:
max_len = 50                    #max length of a tweet

In [38]:
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_len, truncating = 'post')

In [39]:
X_test_tokens = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_tokens, maxlen = max_len, truncating = 'post')

In [40]:
#Defining model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

model = Sequential([
                    Embedding(2000, 64, input_length = max_len),
                    LSTM(64, dropout = 0.2, recurrent_dropout = 0.2),
                    Dense(1, activation = 'sigmoid')
])


In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 64)            128000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 161,089
Trainable params: 161,089
Non-trainable params: 0
_________________________________________________________________


In [42]:
X_train_pad.shape

(1200000, 50)

In [53]:
y_train.shape

(1200000, 1)

In [45]:
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train= y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [46]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.binary_crossentropy,
              metrics = ['accuracy'])

In [48]:
history = model.fit(X_train_pad, y_train, epochs = 1, validation_data=(X_test_pad, y_test))

37500/37500 [==============================] - 2789s 74ms/step - loss: 0.4075 - accuracy: 0.8120 - val_loss: 0.4047 - val_accuracy: 0.8134


In [66]:
def predict_sent(sentence):

  tokens = tokenizer.texts_to_sequences([sentence])
  sentence_pad = pad_sequences(tokens, maxlen=max_len, truncating = 'post')
  pred = model.predict(sentence_pad)
  print(pred)
  if pred <=0.5:
    print('Sentence sentiment is negative')
  else:
    print('Sentence sentiment is positive')

In [67]:
predict_sent('I am happy')

[[0.99172103]]
Sentence sentiment is positive


In [68]:
predict_sent('I am sad')

[[0.00293928]]
Sentence sentiment is negative


In [73]:
predict_sent('I am happy that you got the job, but i will be sad you are going to leave')

[[0.06848097]]
Sentence sentiment is negative
